<a href="https://colab.research.google.com/github/luislizcano/seagrass-mapping-tb/blob/main/Classify/run_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semi-Automated Seagrass Classification Using Earth Engine Python API

This script classify seagrass beds in selected BOA images using ground-data to train the Support Vector Machine classifier. The outputs can be exported to EE Assets. All the training and validation matrices and accuracies can be saved as an Excel file in your (Google Drive) working directory.<br/>


Script by: Luis Lizcano-Sandoval (Modified from [Lizcano-Sandoval et al. 2022](https://linkinghub.elsevier.com/retrieve/pii/S0272771422003924))<br/>
College of Marine Sciences, University of South Florida<br/>
luislizcanos@usf.edu / lsandoval@sig-gis.com<br/>
Updated: 12/26/2024

**Cite as:** *Lizcano-Sandoval, L.; Beck, M.W.; Scolaro, S.; Sherwood, E.T.; Muller-Karger, F. (2025) Cloud-Based Satellite Remote Sensing for Enhancing Seagrass Monitoring and Ecosystem Management (In preparation)*

<font size="4">**Workflow:**</font>

1. Import required images, collections, data, etc.
2. Mask clouds and land
3. Apply Depth-Invariant Index (generates band-ratios B2B3)
4. Sample bands: B1, B2, B3, B4, B/G
5. Train models and classify (SVM)
6. Get confusion matrices and accuracies
7. Export classification images to EE Assets (.tiff)
8. Save matrices in GDrive (.xlxs)

### **Import packages**

In [ ]:
## Authenticate your EE account
import ee
ee.Authenticate()
ee.Initialize(project='INSERT-EE-PROJECT-ID')

In [ ]:
## Clone github repo:
!git clone https://github.com/luislizcano/seagrass-mapping-tb.git

## Install some libraries [may need to restart runtime after]:
!pip install xlsxwriter

In [ ]:
## Run this cell to mount your Google Drive
import os, sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.insert(0,'/content/seagrass-mapping-tb')
sys.path.append('/content/seagrass-mapping-tb/Preprocessing/')

In [ ]:
## Verify you loaded the EE module correctly:
from process import start_processing
print('EE version: ',ee.__version__)

### **Settings:**

In [ ]:
#### Define the source of your image, if from your EE Assets or EE Collections:
# imageSource = 'assets'  ## Read BOA imagery saved in EE Assets (Keeping original file name/id)
imageSource = 'ee'      ## Read BOA imagery from EE Collections

#### Define the type of satellite imagery:
satellite = 'Sentinel2'

#### Settings for running script:
## Classifications grouped by years:
year = '2023'
## Unique code names
segmentCode = 'OTB'
## Name of folder with BOA images in assets (Ignore if imageSource = ee). It can be organized per years:
boaFolder = 'INSERT-ASSETID-BOA-IMAGES/'+year+'/'
## Name of EE folder to save the final output - must create it manually if not exist yet. Can be named "00_RawClassification":
exportFolder = 'INSERT-ASSETID-EXPORT/'+'00_RawClassification/'
## Smooth classified pixels or not? options: '_smooth_' or '_raw_':
smoothStr = '_raw_' ## recommend to use 'raw' and smooth the classified images in the mosaicking step (postprocessing) 

## More metadata for saving image properties:
segmentName = 'Old Tampa Bay' ## Segment name -used as metadata only
regionCountry = 'USA'
state = 'Florida'

### **Insert list of image IDs:**

In [ ]:
## Paste image ids.
## Sentinel 2
imageList = [
'20231219T160659_20231219T160915_T17RLL',
'20231129T160619_20231129T161314_T17RLL'
]

### **Import data:**

In [ ]:
#########################    IMPORT DATA    #########################

# Sandy areas
sandPoly = ee.FeatureCollection("users/lizcanosandoval/TB_seagrass_data/shp/Sand")
print('Sand areas loaded:', sandPoly.get('system:id').getInfo())

# Ground-Points
groundPoints = ee.FeatureCollection('users/lizcanosandoval/TB_seagrass_data/gtp/' + year)
print('Ground-points loaded:', groundPoints.get('system:id').getInfo())

## Landmask
land = ee.Image("users/lizcanosandoval/TB_seagrass_data/Landmask") ##Created from NDWI using Sentinel-2 imagery
print('Land mask loaded:', land.get('system:id').getInfo())

## TB segments:
segments = ee.FeatureCollection("users/lizcanosandoval/TB_seagrass_data/shp/TBEP_Segments_Adapted")
print('Regions loaded:', segments.get('system:id').getInfo())

print('*** Collections loaded correctly! ***')

### **Start classification loop:**

In [ ]:
%%time
start_processing(imageSource,satellite,boaFolder,exportFolder,segments,segmentName,segmentCode,
                 state,regionCountry,imageList,smoothStr,sandPoly,groundPoints,land,1,1,0,0)